# Implementing ETL Pipeline Using SQLAlchemy

SQLAlchemy is a Python module that allows data engineers and developers to define schemas, write queries, and manipulate SQL databases entirely through Python. SQLAlchemy’s Object Relational Mapper (ORM) and Expression Language functionalities allowing us to associate Python classes and constructs with data tables and expressions.

For this project, I will be extracting data from two different mock APIs which are:
- [Random User Generator](https://randomuser.me/) - to get user profiles data
- [JSONPlaceholder](https://jsonplaceholder.typicode.com/) - to get albums and photos data

The extracted data will be in JSON format, which will be transformed into a flat format that can be safely written to the database without error.

It will then be loaded into its database which is `gallery.db`. This database should contain three tables: **users, albums, photos** <br><br>

![Entity Relation Diagram](entity-relation-diagram.png)
<p style="text-align: center;"><i>Entity Relation Diagram for Photo Gallery database</i></p><br>

## Defining a database schema
A database schema defines the structure of a database system, in terms of tables, columns, fields, and the relationships between them. Schemas can be defined in raw SQL, or through the use of SQLAlchemy’s ORM feature.

In [3]:
# Install sqlalchemy first
! pip install sqlalchemy

In [4]:
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import *

engine = create_engine('sqlite:///gallery.db')

Base = declarative_base()

class Users(Base):
    __tablename__ = "users"
    UserId = Column(Integer, primary_key=True)
    Title = Column(String)
    FirstName = Column(String)
    LastName = Column(String)
    Email = Column(String)
    Username = Column(String)
    DOB = Column(DateTime)
    
class Albums(Base):
    __tablename__="albums"
    AlbumId = Column(Integer, primary_key=True)
    UserId = Column(Integer)
    Title = Column(String)
    
class Photos(Base):
    __tablename__="photos"
    PhotoId = Column(Integer, primary_key=True)
    AlbumId = Column(Integer)
    Title = Column(String)
    Url = Column(String)
    
Users.__table__.create(bind=engine, checkfirst=True)
Albums.__table__.create(bind=engine, checkfirst=True)
Photos.__table__.create(bind=engine, checkfirst=True)

## Extract

Once the schema has been defined, the next task is to extract the raw data from its source.
The **Users** table will be populated with profiles randomly generated at `randomuser.me`, while the **Albums** and **Photos** table will contain data courtesy of JSONPlaceholder.

Python’s `Requests` module is used to call these APIs.

In [8]:
import requests

url = 'https://randomuser.me/api/?results=10'
users_json = requests.get(url).json()

url2 = 'https://jsonplaceholder.typicode.com/albums/'
albums_json = requests.get(url2).json()

url3 = 'https://jsonplaceholder.typicode.com/photos/'
photos_json = requests.get(url3).json()

In [9]:
import pprint
pprint.pprint(users_json)

{'info': {'page': 1,
          'results': 10,
          'seed': 'c5e1234a1b8a832a',
          'version': '1.3'},
 'results': [{'cell': '14768453',
              'dob': {'age': 32, 'date': '1989-06-13T10:21:49.240Z'},
              'email': 'josefine.jorgensen@example.com',
              'gender': 'female',
              'id': {'name': 'CPR', 'value': '130689-4231'},
              'location': {'city': 'Odense Sv',
                           'coordinates': {'latitude': '-28.7962',
                                           'longitude': '175.9698'},
                           'country': 'Denmark',
                           'postcode': 34808,
                           'state': 'Hovedstaden',
                           'street': {'name': 'Frederiksberggade',
                                      'number': 619},
                           'timezone': {'description': 'Abu Dhabi, Muscat, '
                                                       'Baku, Tbilisi',
                               

In [10]:
pprint.pprint(albums_json)

[{'id': 1, 'title': 'quidem molestiae enim', 'userId': 1},
 {'id': 2, 'title': 'sunt qui excepturi placeat culpa', 'userId': 1},
 {'id': 3, 'title': 'omnis laborum odio', 'userId': 1},
 {'id': 4, 'title': 'non esse culpa molestiae omnis sed optio', 'userId': 1},
 {'id': 5, 'title': 'eaque aut omnis a', 'userId': 1},
 {'id': 6, 'title': 'natus impedit quibusdam illo est', 'userId': 1},
 {'id': 7, 'title': 'quibusdam autem aliquid et et quia', 'userId': 1},
 {'id': 8, 'title': 'qui fuga est a eum', 'userId': 1},
 {'id': 9, 'title': 'saepe unde necessitatibus rem', 'userId': 1},
 {'id': 10, 'title': 'distinctio laborum qui', 'userId': 1},
 {'id': 11,
  'title': 'quam nostrum impedit mollitia quod et dolor',
  'userId': 2},
 {'id': 12,
  'title': 'consequatur autem doloribus natus consectetur',
  'userId': 2},
 {'id': 13, 'title': 'ab rerum non rerum consequatur ut ea unde', 'userId': 2},
 {'id': 14, 'title': 'ducimus molestias eos animi atque nihil', 'userId': 2},
 {'id': 15,
  'title': '

In [11]:
pprint.pprint(photos_json)

[{'albumId': 1,
  'id': 1,
  'thumbnailUrl': 'https://via.placeholder.com/150/92c952',
  'title': 'accusamus beatae ad facilis cum similique qui sunt',
  'url': 'https://via.placeholder.com/600/92c952'},
 {'albumId': 1,
  'id': 2,
  'thumbnailUrl': 'https://via.placeholder.com/150/771796',
  'title': 'reprehenderit est deserunt velit ipsam',
  'url': 'https://via.placeholder.com/600/771796'},
 {'albumId': 1,
  'id': 3,
  'thumbnailUrl': 'https://via.placeholder.com/150/24f355',
  'title': 'officia porro iure quia iusto qui ipsa ut modi',
  'url': 'https://via.placeholder.com/600/24f355'},
 {'albumId': 1,
  'id': 4,
  'thumbnailUrl': 'https://via.placeholder.com/150/d32776',
  'title': 'culpa odio esse rerum omnis laboriosam voluptate repudiandae',
  'url': 'https://via.placeholder.com/600/d32776'},
 {'albumId': 1,
  'id': 5,
  'thumbnailUrl': 'https://via.placeholder.com/150/f66b97',
  'title': 'natus nisi omnis corporis facere molestiae rerum in',
  'url': 'https://via.placeholder.com

  'id': 135,
  'thumbnailUrl': 'https://via.placeholder.com/150/26016b',
  'title': 'qui harum quis ipsum optio ex',
  'url': 'https://via.placeholder.com/600/26016b'},
 {'albumId': 3,
  'id': 136,
  'thumbnailUrl': 'https://via.placeholder.com/150/2e1c14',
  'title': 'aut voluptas aut temporibus',
  'url': 'https://via.placeholder.com/600/2e1c14'},
 {'albumId': 3,
  'id': 137,
  'thumbnailUrl': 'https://via.placeholder.com/150/41c3dc',
  'title': 'et sit earum praesentium quas quis sint et',
  'url': 'https://via.placeholder.com/600/41c3dc'},
 {'albumId': 3,
  'id': 138,
  'thumbnailUrl': 'https://via.placeholder.com/150/ff79d0',
  'title': 'vitae delectus sed',
  'url': 'https://via.placeholder.com/600/ff79d0'},
 {'albumId': 3,
  'id': 139,
  'thumbnailUrl': 'https://via.placeholder.com/150/ff2fe8',
  'title': 'velit placeat optio corrupti',
  'url': 'https://via.placeholder.com/600/ff2fe8'},
 {'albumId': 3,
  'id': 140,
  'thumbnailUrl': 'https://via.placeholder.com/150/c0798a',
  '

  'url': 'https://via.placeholder.com/600/21f8c2'},
 {'albumId': 5,
  'id': 226,
  'thumbnailUrl': 'https://via.placeholder.com/150/135ce6',
  'title': 'omnis accusantium et',
  'url': 'https://via.placeholder.com/600/135ce6'},
 {'albumId': 5,
  'id': 227,
  'thumbnailUrl': 'https://via.placeholder.com/150/3c1e25',
  'title': 'quae facere aut',
  'url': 'https://via.placeholder.com/600/3c1e25'},
 {'albumId': 5,
  'id': 228,
  'thumbnailUrl': 'https://via.placeholder.com/150/d8b6fa',
  'title': 'laudantium magnam et culpa dolores harum ipsam',
  'url': 'https://via.placeholder.com/600/d8b6fa'},
 {'albumId': 5,
  'id': 229,
  'thumbnailUrl': 'https://via.placeholder.com/150/9d3896',
  'title': 'fugit ut nostrum quia in laborum',
  'url': 'https://via.placeholder.com/600/9d3896'},
 {'albumId': 5,
  'id': 230,
  'thumbnailUrl': 'https://via.placeholder.com/150/b24645',
  'title': 'a deleniti quae exercitationem aut et reprehenderit',
  'url': 'https://via.placeholder.com/600/b24645'},
 {'a

  'id': 361,
  'thumbnailUrl': 'https://via.placeholder.com/150/73a23c',
  'title': 'odio iure cum iusto aut ullam aliquam praesentium',
  'url': 'https://via.placeholder.com/600/73a23c'},
 {'albumId': 8,
  'id': 362,
  'thumbnailUrl': 'https://via.placeholder.com/150/9c8f57',
  'title': 'neque quasi ea quia et',
  'url': 'https://via.placeholder.com/600/9c8f57'},
 {'albumId': 8,
  'id': 363,
  'thumbnailUrl': 'https://via.placeholder.com/150/ee7b2c',
  'title': 'rem sed quam',
  'url': 'https://via.placeholder.com/600/ee7b2c'},
 {'albumId': 8,
  'id': 364,
  'thumbnailUrl': 'https://via.placeholder.com/150/8da619',
  'title': 'et sit repudiandae qui',
  'url': 'https://via.placeholder.com/600/8da619'},
 {'albumId': 8,
  'id': 365,
  'thumbnailUrl': 'https://via.placeholder.com/150/39cac2',
  'title': 'qui officia necessitatibus debitis et sunt quis non minus',
  'url': 'https://via.placeholder.com/600/39cac2'},
 {'albumId': 8,
  'id': 366,
  'thumbnailUrl': 'https://via.placeholder.co

 {'albumId': 9,
  'id': 447,
  'thumbnailUrl': 'https://via.placeholder.com/150/c24531',
  'title': 'et impedit voluptatum',
  'url': 'https://via.placeholder.com/600/c24531'},
 {'albumId': 9,
  'id': 448,
  'thumbnailUrl': 'https://via.placeholder.com/150/d23a91',
  'title': 'et voluptatem animi fuga aut',
  'url': 'https://via.placeholder.com/600/d23a91'},
 {'albumId': 9,
  'id': 449,
  'thumbnailUrl': 'https://via.placeholder.com/150/f65b7a',
  'title': 'dolorem amet architecto aliquam quia quo',
  'url': 'https://via.placeholder.com/600/f65b7a'},
 {'albumId': 9,
  'id': 450,
  'thumbnailUrl': 'https://via.placeholder.com/150/7a530d',
  'title': 'reprehenderit et est qui quo et ad sunt',
  'url': 'https://via.placeholder.com/600/7a530d'},
 {'albumId': 10,
  'id': 451,
  'thumbnailUrl': 'https://via.placeholder.com/150/5e912a',
  'title': 'dolorem accusantium corrupti incidunt quas ex est',
  'url': 'https://via.placeholder.com/600/5e912a'},
 {'albumId': 10,
  'id': 452,
  'thumbnail

...

  'url': 'https://via.placeholder.com/600/b4750d'},
 {'albumId': 93,
  'id': 4622,
  'thumbnailUrl': 'https://via.placeholder.com/150/b20270',
  'title': 'tempore quis sed deleniti non temporibus',
  'url': 'https://via.placeholder.com/600/b20270'},
 {'albumId': 93,
  'id': 4623,
  'thumbnailUrl': 'https://via.placeholder.com/150/beb35f',
  'title': 'maxime minima ea perferendis illum magni enim sed velit',
  'url': 'https://via.placeholder.com/600/beb35f'},
 {'albumId': 93,
  'id': 4624,
  'thumbnailUrl': 'https://via.placeholder.com/150/e41c4a',
  'title': 'nihil eligendi asperiores corrupti ipsum vel dicta eos iure',
  'url': 'https://via.placeholder.com/600/e41c4a'},
 {'albumId': 93,
  'id': 4625,
  'thumbnailUrl': 'https://via.placeholder.com/150/5c7f07',
  'title': 'praesentium sunt quidem velit repudiandae beatae',
  'url': 'https://via.placeholder.com/600/5c7f07'},
 {'albumId': 93,
  'id': 4626,
  'thumbnailUrl': 'https://via.placeholder.com/150/e1699f',
  'title': 'est odit qu

  'url': 'https://via.placeholder.com/600/22e6f4'},
 {'albumId': 94,
  'id': 4699,
  'thumbnailUrl': 'https://via.placeholder.com/150/82d058',
  'title': 'excepturi tempore at voluptatem eaque omnis impedit fuga neque',
  'url': 'https://via.placeholder.com/600/82d058'},
 {'albumId': 94,
  'id': 4700,
  'thumbnailUrl': 'https://via.placeholder.com/150/4738f4',
  'title': 'consectetur molestias veritatis',
  'url': 'https://via.placeholder.com/600/4738f4'},
 {'albumId': 95,
  'id': 4701,
  'thumbnailUrl': 'https://via.placeholder.com/150/29dd1f',
  'title': 'eum reprehenderit vel',
  'url': 'https://via.placeholder.com/600/29dd1f'},
 {'albumId': 95,
  'id': 4702,
  'thumbnailUrl': 'https://via.placeholder.com/150/7e4f06',
  'title': 'temporibus minus occaecati nostrum quia ut alias',
  'url': 'https://via.placeholder.com/600/7e4f06'},
 {'albumId': 95,
  'id': 4703,
  'thumbnailUrl': 'https://via.placeholder.com/150/9d3d94',
  'title': 'suscipit tempore non placeat nihil pariatur vel omn

  'url': 'https://via.placeholder.com/600/93ef92'},
 {'albumId': 96,
  'id': 4761,
  'thumbnailUrl': 'https://via.placeholder.com/150/2a64a7',
  'title': 'et neque voluptatum rerum est necessitatibus',
  'url': 'https://via.placeholder.com/600/2a64a7'},
 {'albumId': 96,
  'id': 4762,
  'thumbnailUrl': 'https://via.placeholder.com/150/7240c7',
  'title': 'iure sapiente aspernatur velit alias sint',
  'url': 'https://via.placeholder.com/600/7240c7'},
 {'albumId': 96,
  'id': 4763,
  'thumbnailUrl': 'https://via.placeholder.com/150/8ea5ed',
  'title': 'tempora qui quia porro omnis modi laudantium',
  'url': 'https://via.placeholder.com/600/8ea5ed'},
 {'albumId': 96,
  'id': 4764,
  'thumbnailUrl': 'https://via.placeholder.com/150/882179',
  'title': 'nostrum tempore dolores ut recusandae eos debitis sequi optio',
  'url': 'https://via.placeholder.com/600/882179'},
 {'albumId': 96,
  'id': 4765,
  'thumbnailUrl': 'https://via.placeholder.com/150/5315c6',
  'title': 'unde quod molestias',
 

 {'albumId': 98,
  'id': 4888,
  'thumbnailUrl': 'https://via.placeholder.com/150/9e8e55',
  'title': 'modi necessitatibus voluptatem sed facilis qui ipsam nulla',
  'url': 'https://via.placeholder.com/600/9e8e55'},
 {'albumId': 98,
  'id': 4889,
  'thumbnailUrl': 'https://via.placeholder.com/150/26141f',
  'title': 'quam aut libero temporibus',
  'url': 'https://via.placeholder.com/600/26141f'},
 {'albumId': 98,
  'id': 4890,
  'thumbnailUrl': 'https://via.placeholder.com/150/415f05',
  'title': 'in ad sit consequatur est et adipisci',
  'url': 'https://via.placeholder.com/600/415f05'},
 {'albumId': 98,
  'id': 4891,
  'thumbnailUrl': 'https://via.placeholder.com/150/d230a4',
  'title': 'et qui dolores sunt',
  'url': 'https://via.placeholder.com/600/d230a4'},
 {'albumId': 98,
  'id': 4892,
  'thumbnailUrl': 'https://via.placeholder.com/150/e7b15a',
  'title': 'eos nam quas qui eum consequatur dolorum veniam',
  'url': 'https://via.placeholder.com/600/e7b15a'},
 {'albumId': 98,
  'id'

  'url': 'https://via.placeholder.com/600/6e9a1a'},
 {'albumId': 100,
  'id': 4979,
  'thumbnailUrl': 'https://via.placeholder.com/150/290132',
  'title': 'unde aliquam totam qui corrupti sed',
  'url': 'https://via.placeholder.com/600/290132'},
 {'albumId': 100,
  'id': 4980,
  'thumbnailUrl': 'https://via.placeholder.com/150/1f913a',
  'title': 'et dolores provident sint',
  'url': 'https://via.placeholder.com/600/1f913a'},
 {'albumId': 100,
  'id': 4981,
  'thumbnailUrl': 'https://via.placeholder.com/150/f79a53',
  'title': 'officiis veniam adipisci repudiandae asperiores ut rerum sint',
  'url': 'https://via.placeholder.com/600/f79a53'},
 {'albumId': 100,
  'id': 4982,
  'thumbnailUrl': 'https://via.placeholder.com/150/4d611d',
  'title': 'blanditiis doloremque deleniti quo voluptatem odio nobis ut et',
  'url': 'https://via.placeholder.com/600/4d611d'},
 {'albumId': 100,
  'id': 4983,
  'thumbnailUrl': 'https://via.placeholder.com/150/32a201',
  'title': 'et nesciunt ducimus quis 

The data is currently held in three objects (`users_json`,`albums_json`, and `photos_json`) in JSON format. The next step will be to transform and load this data into the tables defined earlier.

## Transform

Transform the data from its current nested JSON format to a flat format that can be safely written to the database without error.
The code below creates three lists, `users`,`albums` and `photos`, which will be used in the final step:

In [12]:
from datetime import datetime, timedelta
import dateutil.parser

users, albums, photos = [],[],[]

for i, result in enumerate(users_json['results']):
    row = {}
    row['UserId'] = i
    row['Title'] = result['name']['title']
    row['FirstName'] = result['name']['first']
    row['LastName'] = result['name']['last']
    row['Email'] = result['email']
    row['Username'] = result['login']['username']
    d = dateutil.parser.parse(result['dob']['date'])
    dob = d.strftime('%Y-%m-%d %H:%M:&S')
    dob = datetime.strptime(dob, '%Y-%m-%d %H:%M:&S' )
    row['DOB']=dob
    users.append(row)

for result in albums_json:
    row = {}
    row['AlbumId'] = result['id']
    row['UserId'] = result['userId']
    row['Title'] = result['title']
    albums.append(row)
    
for result in photos_json:
    row = {}
    row['PhotoId'] = result['id']
    row['AlbumId'] = result['albumId']
    row['Title'] = result['title']
    row['Url'] = result['url']
    photos.append(row)

In [13]:
print(users)

[{'UserId': 0, 'Title': 'Miss', 'FirstName': 'Josefine', 'LastName': 'Jørgensen', 'Email': 'josefine.jorgensen@example.com', 'Username': 'happyladybug369', 'DOB': datetime.datetime(1989, 6, 13, 10, 21)}, {'UserId': 1, 'Title': 'Mrs', 'FirstName': 'Frida', 'LastName': 'Riise', 'Email': 'frida.riise@example.com', 'Username': 'ticklishmouse475', 'DOB': datetime.datetime(1958, 3, 28, 19, 53)}, {'UserId': 2, 'Title': 'Mr', 'FirstName': 'Thomas', 'LastName': 'Kelley', 'Email': 'thomas.kelley@example.com', 'Username': 'heavylion909', 'DOB': datetime.datetime(1973, 8, 14, 10, 29)}, {'UserId': 3, 'Title': 'Mr', 'FirstName': 'اميرحسين', 'LastName': 'نجاتی', 'Email': 'myrhsyn.njty@example.com', 'Username': 'heavyfrog421', 'DOB': datetime.datetime(1996, 9, 14, 18, 9)}, {'UserId': 4, 'Title': 'Monsieur', 'FirstName': 'Grégoire', 'LastName': 'Perez', 'Email': 'gregoire.perez@example.com', 'Username': 'happycat530', 'DOB': datetime.datetime(1979, 6, 3, 15, 56)}, {'UserId': 5, 'Title': 'Ms', 'FirstNam

In [14]:
print(albums)

[{'AlbumId': 1, 'UserId': 1, 'Title': 'quidem molestiae enim'}, {'AlbumId': 2, 'UserId': 1, 'Title': 'sunt qui excepturi placeat culpa'}, {'AlbumId': 3, 'UserId': 1, 'Title': 'omnis laborum odio'}, {'AlbumId': 4, 'UserId': 1, 'Title': 'non esse culpa molestiae omnis sed optio'}, {'AlbumId': 5, 'UserId': 1, 'Title': 'eaque aut omnis a'}, {'AlbumId': 6, 'UserId': 1, 'Title': 'natus impedit quibusdam illo est'}, {'AlbumId': 7, 'UserId': 1, 'Title': 'quibusdam autem aliquid et et quia'}, {'AlbumId': 8, 'UserId': 1, 'Title': 'qui fuga est a eum'}, {'AlbumId': 9, 'UserId': 1, 'Title': 'saepe unde necessitatibus rem'}, {'AlbumId': 10, 'UserId': 1, 'Title': 'distinctio laborum qui'}, {'AlbumId': 11, 'UserId': 2, 'Title': 'quam nostrum impedit mollitia quod et dolor'}, {'AlbumId': 12, 'UserId': 2, 'Title': 'consequatur autem doloribus natus consectetur'}, {'AlbumId': 13, 'UserId': 2, 'Title': 'ab rerum non rerum consequatur ut ea unde'}, {'AlbumId': 14, 'UserId': 2, 'Title': 'ducimus molestias 

In [15]:
print(photos)

[{'PhotoId': 1, 'AlbumId': 1, 'Title': 'accusamus beatae ad facilis cum similique qui sunt', 'Url': 'https://via.placeholder.com/600/92c952'}, {'PhotoId': 2, 'AlbumId': 1, 'Title': 'reprehenderit est deserunt velit ipsam', 'Url': 'https://via.placeholder.com/600/771796'}, {'PhotoId': 3, 'AlbumId': 1, 'Title': 'officia porro iure quia iusto qui ipsa ut modi', 'Url': 'https://via.placeholder.com/600/24f355'}, {'PhotoId': 4, 'AlbumId': 1, 'Title': 'culpa odio esse rerum omnis laboriosam voluptate repudiandae', 'Url': 'https://via.placeholder.com/600/d32776'}, {'PhotoId': 5, 'AlbumId': 1, 'Title': 'natus nisi omnis corporis facere molestiae rerum in', 'Url': 'https://via.placeholder.com/600/f66b97'}, {'PhotoId': 6, 'AlbumId': 1, 'Title': 'accusamus ea aliquid et amet sequi nemo', 'Url': 'https://via.placeholder.com/600/56a8c2'}, {'PhotoId': 7, 'AlbumId': 1, 'Title': 'officia delectus consequatur vero aut veniam explicabo molestias', 'Url': 'https://via.placeholder.com/600/b0f7cc'}, {'Photo

## Load

Finally, the data is in a form that can be loaded into the database. SQLAlchemy makes this step straightforward through its Session API.
The code below creates a new session object, adds rows to it, then merges and commits them to the database:

In [16]:
Session = sessionmaker(bind=engine)

session = Session()  

for user in users:
    row = Users(**user)
    session.add(row)
    
for album in albums:
    row = Albums(**album)
    session.add(row)
    
for photo in photos:
    row = Photos(**photo)
    session.add(row)
    
session.commit()

## Check existing tables in database

In [17]:
%load_ext sql
%sql sqlite:///gallery.db

In [23]:
%sql SELECT count(*) FROM users;

 * sqlite:///gallery.db
Done.


count(*)
10


In [24]:
%sql SELECT * FROM users LIMIT 5;

 * sqlite:///gallery.db
Done.


UserId,Title,FirstName,LastName,Email,Username,DOB
0,Miss,Josefine,Jørgensen,josefine.jorgensen@example.com,happyladybug369,1989-06-13 10:21:00.000000
1,Mrs,Frida,Riise,frida.riise@example.com,ticklishmouse475,1958-03-28 19:53:00.000000
2,Mr,Thomas,Kelley,thomas.kelley@example.com,heavylion909,1973-08-14 10:29:00.000000
3,Mr,اميرحسين,نجاتی,myrhsyn.njty@example.com,heavyfrog421,1996-09-14 18:09:00.000000
4,Monsieur,Grégoire,Perez,gregoire.perez@example.com,happycat530,1979-06-03 15:56:00.000000


In [22]:
%sql SELECT count(*) FROM albums;

 * sqlite:///gallery.db
Done.


count(*)
100


In [25]:
%sql SELECT * FROM albums LIMIT 5;

 * sqlite:///gallery.db
Done.


AlbumId,UserId,Title
1,1,quidem molestiae enim
2,1,sunt qui excepturi placeat culpa
3,1,omnis laborum odio
4,1,non esse culpa molestiae omnis sed optio
5,1,eaque aut omnis a


In [21]:
%sql SELECT count(*) FROM photos;

 * sqlite:///gallery.db
Done.


count(*)
5000


In [26]:
%sql SELECT * FROM photos LIMIT 5;

 * sqlite:///gallery.db
Done.


PhotoId,AlbumId,Title,Url
1,1,accusamus beatae ad facilis cum similique qui sunt,https://via.placeholder.com/600/92c952
2,1,reprehenderit est deserunt velit ipsam,https://via.placeholder.com/600/771796
3,1,officia porro iure quia iusto qui ipsa ut modi,https://via.placeholder.com/600/24f355
4,1,culpa odio esse rerum omnis laboriosam voluptate repudiandae,https://via.placeholder.com/600/d32776
5,1,natus nisi omnis corporis facere molestiae rerum in,https://via.placeholder.com/600/f66b97


The End.